In [535]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from game import Game
from player import Player

In [590]:
from Cyganik_Chwilczynski import CyganikChwilczynski

In [537]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [538]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())
print(len(getDeck()))

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]
24


In [539]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(9, 0), (12, 1), (9, 3), (12, 0), (9, 2), (12, 3), (13, 3), (10, 3)]
[(13, 1), (11, 0), (13, 0), (14, 0), (11, 2), (14, 3), (13, 2), (14, 2)]


In [540]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [593]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    # print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 100
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                # print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [594]:
from KacperTrebacz145453Player import KacperTrebacz145453
from checker import Checker
from lier import Lier
from lier_checker import LierChecker

In [595]:
strategy = [
            ["CyganikChwilczynski", "CyganikChwilczynski", CyganikChwilczynski],
            ["RandomPlayer", "RANDOM", RandomPlayer],
            ["KacperTrebacz145453", "KacperTrebacz145453", KacperTrebacz145453],
            ["Lier", "Lier", Lier],
            ["Checker", "Checker", Checker],
            ["LierChecker", "LierChecker", LierChecker],
           ]

In [616]:
import pandas as pd

match_results = pd.DataFrame(columns = ["Player 1", "Player 2", "Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"])

for A in range(len(strategy)-1):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        # add a row to the dataframe
        match_results.loc[len(match_results)] = {"Player 1": strategy[A][1], "Player 2": strategy[B][1], "Wins": results[0][0], "Draws": results[1][0], "Moves": results[2][0], "Cards": results[3][0], "Pile Size": results[4], "Checks": results[5][0], "Draw Decisions": results[6][0], "Cheats": results[7][0], "Errors": results[8][0], "Total errors": results[9]}

match_results

==== 0 CyganikChwilczynski
1 RandomPlayer
2 KacperTrebacz145453
3 Lier
4 Checker
5 LierChecker
==== 1 RandomPlayer
2 KacperTrebacz145453
3 Lier
4 Checker
5 LierChecker
==== 2 KacperTrebacz145453
3 Lier
4 Checker
5 LierChecker
==== 3 Lier
4 Checker
5 LierChecker
==== 4 Checker
5 LierChecker


,Player 1,Player 2,Wins,Draws,Moves,Cards,Pile Size,Checks,Draw Decisions,Cheats,Errors,Total errors
0,CyganikChwilczynski,RANDOM,71,0,19.83,1.40,6.16,0.97,1.20,5.94,0,0
1,CyganikChwilczynski,KacperTrebacz145453,57,0,11.35,2.49,7.83,0.98,0.36,1.69,0,0
2,CyganikChwilczynski,Lier,52,0,8.87,1.75,12.32,0.38,0.37,0.73,0,0
3,CyganikChwilczynski,Checker,83,0,9.69,0.58,8.86,0.00,0.26,0.62,0,0
4,CyganikChwilczynski,LierChecker,84,0,9.45,0.81,8.13,0.07,0.25,0.59,0,0
5,RANDOM,KacperTrebacz145453,3,0,12.87,13.92,1.64,6.68,0.01,7.66,0,0
6,RANDOM,Lier,25,0,20.09,9.12,5.64,9.51,0.11,11.07,0,0
7,RANDOM,Checker,1,0,16.38,14.10,1.74,7.38,0.01,8.93,0,0
8,RANDOM,LierChecker,2,0,15.66,13.84,2.03,7.30,0.02,8.97,0,0
9,KacperTrebacz145453,Lier,52,0,12.54,4.82,8.17,3.89,0.00,2.70,0,0


In [ ]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 KacperTrebacz145453
2 YourName
==== 1 KacperTrebacz145453
2 YourName
==== 2 YourName


In [529]:
full_results

[[None,
  [[75, 925],
   [0, 0],
   [12.945, 13.392],
   [13.336, 0.925],
   1.739,
   [6.644, 3.923],
   [0.035, 0.018],
   [7.661, 3.606],
   [0, 0],
   0,
   0],
  [[174, 826],
   [40, 40],
   [27.495, 27.793],
   [10.799, 2.143],
   3.058,
   [10.497, 1.552],
   [0.052, 6.827],
   [14.646, 0.011],
   [0, 0],
   0,
   40]],
 [None,
  None,
  [[471, 529],
   [0, 0],
   [11.578, 11.621],
   [5.118, 4.441],
   6.441,
   [2.13, 0.933],
   [0.003, 2.013],
   [2.643, 0.087],
   [0, 0],
   0,
   0]],
 [None, None, None]]

Simple stats

In [530]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 75 174 
1
KacperTrebacz145453 925 - 471 
2
YourName 826 529 - 
Draws
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 0 40 
1
KacperTrebacz145453 0 - 0 
2
YourName 40 0 - 
Moves
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 12.945 27.495 
1
KacperTrebacz145453 13.392 - 11.578 
2
YourName 27.793 11.621 - 
Cards
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 13.336 10.799 
1
KacperTrebacz145453 0.925 - 5.118 
2
YourName 2.143 4.441 - 
Checks
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 6.644 10.497 
1
KacperTrebacz145453 3.923 - 2.13 
2
YourName 1.552 0.933 - 
Draw Decisions
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 0.035 0.052 
1
KacperTrebacz145453 0.018 - 0.003 
2
YourName 6.827 2.013 - 
Cheats
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 7.661 14.646 
1
KacperTrebacz145453 3.606 - 2.643 
2
YourName 0.011 0.087 - 
Errors
 RANDOM KacperTrebacz145453 YourName 
0
RANDOM - 0 0 
1
KacperTrebacz145453 0 - 0 
2
YourNam